In [1]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
import numpy as np

In [2]:
height = 266
width = 416
batch_size = 32

In [3]:
train_data_dir = '/content/drive/MyDrive/music_split_data/train'
test_data_dir = '/content/drive/MyDrive/music_split_data/test'
val_data_dir = '/content/drive/MyDrive/music_split_data/val'

In [5]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    validation_split=0.4)
train_data = data_generator.flow_from_directory(train_data_dir,
                                                target_size=(height,width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training')
val_data = data_generator.flow_from_directory(train_data_dir,
                                                target_size=(height,width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation')
test_data = data_generator.flow_from_directory(train_data_dir,
                                                target_size=(height,width),
                                                batch_size=1,
                                                class_mode='categorical',
                                                subset='validation')

Found 2158 images belonging to 10 classes.
Found 1438 images belonging to 10 classes.
Found 1438 images belonging to 10 classes.


In [6]:
x,y = train_data.next()
x.shape

(32, 266, 416, 3)

In [8]:
base_model = ResNet50(include_top=False,weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
prediction = Dense(10,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=prediction)

for layer in base_model.layers:
  layer.trainable = False

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_data,epochs=10)

Epoch 1/10
68/68 [==============================] - 452s 6s/step - loss: 1.7078 - accuracy: 0.4713
Epoch 2/10
68/68 [==============================] - 72s 1s/step - loss: 1.0776 - accuracy: 0.6219
Epoch 3/10
68/68 [==============================] - 72s 1s/step - loss: 0.8873 - accuracy: 0.6969
Epoch 4/10
68/68 [==============================] - 72s 1s/step - loss: 0.8178 - accuracy: 0.7229
Epoch 5/10
68/68 [==============================] - 72s 1s/step - loss: 0.7455 - accuracy: 0.7340
Epoch 6/10
68/68 [==============================] - 72s 1s/step - loss: 0.7013 - accuracy: 0.7456
Epoch 7/10
68/68 [==============================] - 72s 1s/step - loss: 0.6352 - accuracy: 0.7850
Epoch 8/10
68/68 [==============================] - 72s 1s/step - loss: 0.6831 - accuracy: 0.7567
Epoch 9/10
68/68 [==============================] - 72s 1s/step - loss: 0.5983 - accuracy: 0.7943
Epoch 10/10
68/68 [==============================] - 71s 1s/step - loss: 0.5650 - accuracy: 0.8044


In [9]:
# testing model
test_loss, test_acc = model.evaluate(test_data,verbose=2)
print(f'test accuracy is {100*test_acc}%')

1438/1438 - 289s - loss: 1.5517 - accuracy: 0.5341
test accuracy is 53.40750813484192%


In [ ]:
# code for confusion matrix